In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv
import re
import random
k='2014-01-01'
j='2016-12-31'
dataa=[]
import datetime
dates = []
dt = datetime.datetime.strptime(k, "%Y-%m-%d")
date = k[:]
while date <= j:
    dates.append(date)
    dt = dt + datetime.timedelta(1)
    date = dt.strftime("%Y-%m-%d")

def ua():
    """随机获取一个浏览器用户信息"""

    user_agents = [
        'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
        'Opera/9.25 (Windows NT 5.1; U; en)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
        'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
        'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
        'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
        'Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7',
        'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0',

    ]

    agent = random.choice(user_agents)

    return agent

for wor in ['共享单车']:
    for tii in dates:  # 循环这一年的每天
        print(tii)
        for i in range(1, 2):#因为微博最多只能50页
            url = f'https://s.weibo.com/weibo?q={wor}&typeall=1&suball=1&timescope=custom:{tii}-0:{tii}-23&Refer=g&page={i}'            
            #网站url
            headers = {
                'cookie':'_T_WM=79452191668; XSRF-TOKEN=25c18c; WEIBOCN_FROM=1110005030; mweibo_short_token=f102c0049e; SCF=ApWp3d4Y88M0WvYCVhkgsasd3T9bHbfPd9ad5Z1MGFxfBpShVDmO-9MlABHdR9QWTo8c5jKwWUZ0BkZsMKKSRMQ.; SUB=_2A25IFprGDeRhGeFG6FYR8S7Owz-IHXVr-CaOrDV6PUJbktANLVOikW1NeauaUIGumaJdfFTuzOR3SKpyKLn2bepw; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFaFL9wsf1sc9812OKZcN525JpX5K-hUgL.FoMRe0B7eK5E1he2dJLoI0YLxKnL1h2L1-2LxKnLBoBLBKBLxKqLBK-LB.eLxKML1-2L1hBLxKML1heLBKMLxK.LBo.L1h-LxK.LBK-LB-Bt; SSOLoginState=1695738518; ALF=1698330518; MLOGIN=1; M_WEIBOCN_PARAMS=uicode%3D20000174',
                'user-agent': 'Chrome/98.0.4758.102 Safari/537.36'
                #'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
            }#请求头
            
            
            sleep(1)#停顿1s
            #print('1秒后继续采集')
            while 1:
                try:
                    html0 = requests.get(url=url, headers=headers).text
                    break
                except:
                    pass
            #发起请求
            t = BeautifulSoup(html0, 'lxml')
            #bs4数据解析
            card = t.find_all('div', class_="card-wrap")
            print(len(card))
            print(card)
            
            #定位没个博文的位置
            try:
                pd1 = t.find('div', class_="card card-no-result s-pt20b40")#如果找没有信息到直接跳出程序
                if pd1!=None:
                    continue
                    #break
            except:
                pd1=''
            for p in card:
                dic={}
                if p.find_all('p', class_="txt")!=[]:
                    print(p.find_all('p', class_="txt"))
                    try:
                        dic['word']=wor
                        yh_url='https:'+p.find('div',class_="avator").a['href']
                        dic['user_url']=yh_url
                        bw_url='https:'+p.find('div', class_="from").a['href']
                        
                        dic['url']=bw_url
                        yh_id=bw_url.split('/')[3]
                        dic['user_id']=yh_id
                        bo_id=bw_url.split('/')[4].replace('?refer_flag=1001030103_','')
                        dic['id']=bo_id
                        id=yh_id+'_'+bo_id
                        dic['_id']=id
                        txt=p.find_all('p', class_="txt")[-1].find_all('a')
                        dic['place']='无'#新增
                        
                        for tx in txt:
                            try:
                                if tx.find('i',class_="wbicon").text=='2':
                                    dic['place']=str(tx.text).replace('2','')
                            except:
                                pass
                        
                        dic['content'] = str(p.find_all('p', class_="txt")[-1].text).replace('\n', '').replace('\u200b','').replace('收起d','')#博文内容
                        dic['comment']= str(p.find_all('a', class_="woo-box-flex woo-box-alignCenter woo-box-justifyCenter")[-2].text).replace(' ','')
                        #dic['comment'] = str(p.find_all(text=re.compile("评论 .*?"))[0]).replace(' ','').replace('评论','')#博文评论
                        if dic['comment']== '评论':
                            dic['comment']= 0
                        dic['name']= p.find('a', class_="name").text
                        time = str(p.find('div', class_="from").text).replace(' ', '').replace('\n', '').replace('\xa0','')#发博时间
                        time = time + '来自0'
                        dic['pubtime']= time.split('来自')[0]
                        dic['tools']=time.split('来自')[1]
                        dic['like']= str(p.find_all('span', class_="woo-like-count")[-1].text).replace(' ', '')                    
                        if dic['like']== '赞':
                            dic['like']= 0
                        dic['transfer']= str(p.find_all('a', class_="woo-box-flex woo-box-alignCenter woo-box-justifyCenter")[-3].text).replace(' ', '')
                         
                        if dic['transfer']== '转发':
                            dic['transfer']= 0
                        yhurl = f'https://weibo.com/ajax/profile/info?uid={yh_id}'
                        while 1:
                            try:
                                uhht = requests.get(url=yhurl, headers=headers).json()
                                break
                            except:
                                pass
                        print(uhht['data']['user'])
                        dic['num_follows'] = uhht['data']['user']['friends_count']
                        dic['num_fans'] = uhht['data']['user']['followers_count']
                        dic['num_tweets'] = uhht['data']['user']['statuses_count']
                        xb = uhht['data']['user']['gender']

                        if xb == 'f':
                            dic['gender'] = '女'
                        else:
                            dic['gender'] = '男'
                        dic['location'] = uhht['data']['user']['location']
                        dic['verified']= uhht['data']['user']['verified']
                        dic['verified_type']= uhht['data']['user']['verified_type']
                        try:
                            dic['verified_reason'] = uhht['data']['user']['verified_reason']
                        except:
                            dic['verified_reason'] =''
                            pass
                        
                        print(f"成功写入：{dic}")
                    except:
                        print(i,"失败")
                        pass
                    
print("完成")